## wordExtract.ipynb

### Purpose: Extract OLE Embedded files from a Word document

#### Author: Chris Maderia 

SOURCES: \
https://stackoverflow.com/questions/59003527/extract-acrobat-document-object-from-a-table-in-docx-file-in-python \
https://stackoverflow.com/questions/58254609/python-docx-parse-a-table-to-panda-dataframe

### OLE identifiers  (in ole.listdir)
CONTENTS = <b>PDF</b> \
EPRINT = <b>ZIP</b> \
Microsoft_Word_Document = <b>Word</b> docx (not an ole file)\
Microsoft_Excel_Worksheet = <b>Excel</b> xlsx (not an ole file)\
Microsoft_Word_97 - 2003_Document = <b>Word 97-2003</b> doc (not an ole file)\
Microsoft_Excel_97 - 2003_Worksheet = <b>Excel 97-2003</b> xls (not an ole file)

In [1]:
# Import packages
import os
from docx import Document
from docx.opc.constants import RELATIONSHIP_TYPE as RT
import pandas as pd
import olefile
from zipfile import ZipFile
from glob import glob
from io import BytesIO, StringIO
import zipfile
import struct
import tarfile

In [2]:
# path where your word doc(s) live (should be in a separate folder)
in_root = r'G:\Projects\BusDev\BD_PFAS\ECHA_PFAS\Word_Extract\downloaded'

# location to save extracted files
out_root = r'G:\Projects\BusDev\BD_PFAS\ECHA_PFAS\Word_Extract\extracted\testing'

In [3]:
# Pull pdfs from word document
# Four known file types (so far): Word, Excel, Zip, PDF

# Loop through all the .docx files in the folder
for filename in glob(os.path.join(in_root, '*.docx')):
    # Create new folder to store each Word doc's files
    base_name = os.path.basename(filename).split('.')[0]
    out_dir = os.path.join(out_root, base_name)
    os.mkdir(out_dir)

    
    # --------- BEGIN ID PART --------------
    # (Note: some Document IDs may NOT match actual Attachment IDs due to differences in naming)
    id_list = []
    document = Document(filename)
    tables = []
    # create data frame of all the tables in the doc
    for table in document.tables:
        df = [['' for i in range(len(table.columns))] for j in range(len(table.rows))]
        for i, row in enumerate(table.rows):
            for j, cell in enumerate(row.cells):
                if cell.text:
                    df[i][j] = cell.text
        tables.append(pd.DataFrame(df))
    # get id list of all tables with Attachments existing (not redacted)
    for t in tables:
        # Attachment but not "Attachment Redacted"
        if ((True in list(t.stack().str.contains('Attachment', case=False).unstack().any(axis=1))) and (True not in list(t.stack().str.contains('Attachment:\n<redacted>', case=False).unstack().any(axis=1)))):
            try:
                id_list.append((t[0][1]))
            # if getting row 1 doesn't work, then try row 0 
            except:
                id_list.append((t[0][0]))

    # Try to open the document as ZIP file
    entry_list = []
    with ZipFile(filename, "r") as zip:

        # Find files in the word/embeddings folder of the ZIP file
        for entry in zip.infolist():
            if entry.filename.startswith("word/embeddings/"):
                entry_list.append(entry)

    # GET THE LIST IN PROPER ORDER
    # get all oleObjects/OR other objects leading up to (but not including) oleObject1 
    subset = entry_list[0:entry_list.index([e for e in entry_list if 'oleObject1.' in e.filename][0])]

    # get the remainder of the object list
    entry_list = entry_list[entry_list.index([e for e in entry_list if 'oleObject1.' in e.filename][0]):len(entry_list)]

    # append the subset to the end of the object list
    entry_list.extend(subset)
    # ----- END ID PART ------------

    
    # ----- BEGIN EMBEDDED FILE EXTRACTION -----------
    # find the embedded files and perform the extraction             
    with ZipFile(filename, "r") as zip:
        for entry in entry_list:
            # Naming
            ole_id = int(id_list[entry_list.index(entry)])
            docnum = int(''.join([str(s) for s in filename.split('.doc')[0] if s.isdigit()]))
            # Try to open the embedded OLE file
            with zip.open(entry.filename) as f:
                if 'Microsoft_Word_Document' in f.name:
                    # extract the doc
                    word_filename = "Part%i"%docnum + "_" + "Document_%i.docx" %ole_id
                    word_path = os.path.join(out_dir, word_filename)
                    with open(word_path, 'wb') as word_ref:
                        word_ref.write(f.read())
                if 'Microsoft_Excel_Worksheet' in f.name:
                    # extract the spreadsheet (blank issue)
                    excel_filename = "Part%i"%docnum + "_" + "Document_%i.xlsx" %ole_id
                    excel_path = os.path.join(out_dir, excel_filename)
                    with open(excel_path, 'wb') as excel_ref:
                        excel_ref.write(f.read())
                if 'Microsoft_Word_97_-_2003_Document' in f.name:
                    # extract the doc
                    worde_filename = "Part%i"%docnum + "_" + "Document_%i.doc" %ole_id
                    worde_path = os.path.join(out_dir, worde_filename)
                    with open(worde_path, 'wb') as worde_ref:
                        worde_ref.write(f.read())
                if 'Microsoft_Excel_97_-_2003_Worksheet' in f.name:
                    # extract the spreadsheet (blank issue)
                    excele_filename = "Part%i"%docnum + "_" + "Document_%i.xls" %ole_id
                    excele_path = os.path.join(out_dir, excele_filename)
                    with open(excele_path, 'wb') as excele_ref:
                        excele_ref.write(f.read())
                        
                if 'oleObject' in f.name:
                    ole = olefile.OleFileIO(f)  #, write_mode=True)
                    # PDF files
                    if 'CONTENTS' in str(ole.listdir()):  # in str
                        pdf_data = ole.openstream('CONTENTS').read()
                        # naming
                        pdf_filename = "Part%i"%docnum + "_" + "Document_%i.pdf" %ole_id

                        # Save the PDF
                        pdfpath = os.path.join(out_dir, pdf_filename)
                        with open(pdfpath, 'wb') as out_pdf:
                          out_pdf.write(pdf_data)
                            
                    # ZIP files (only openable from the 7-zip File Manager)
                    if 'EPRINT' in str(ole.listdir()):
                        zip_filename = "Part%i"%docnum + "_" + "Document_%i" %ole_id
    
                        # create folder to store the extracted zip files
                        zipdir = os.path.join(out_dir, zip_filename)

                        with open(zipdir, 'wb') as zip_ref:
                            zip_ref.write(f.read())

    print(base_name)

rest_pfas_rcom_part100_en
rest_pfas_rcom_part101_en
rest_pfas_rcom_part102_en
rest_pfas_rcom_part103_en
rest_pfas_rcom_part104_en
rest_pfas_rcom_part105_en
rest_pfas_rcom_part106_en
rest_pfas_rcom_part107_en
rest_pfas_rcom_part108_en
rest_pfas_rcom_part109_en
rest_pfas_rcom_part10_en
rest_pfas_rcom_part110_en
rest_pfas_rcom_part111_en
rest_pfas_rcom_part112_en
rest_pfas_rcom_part113_en
rest_pfas_rcom_part114_en
rest_pfas_rcom_part115_en
rest_pfas_rcom_part116_en
rest_pfas_rcom_part117_en
rest_pfas_rcom_part118_en
rest_pfas_rcom_part119_en
rest_pfas_rcom_part11_en
rest_pfas_rcom_part120_en
rest_pfas_rcom_part121_en
rest_pfas_rcom_part122_en
rest_pfas_rcom_part123_en
rest_pfas_rcom_part12_en
rest_pfas_rcom_part13_en
rest_pfas_rcom_part14_en
rest_pfas_rcom_part15_en
rest_pfas_rcom_part16_en
rest_pfas_rcom_part17_en
rest_pfas_rcom_part18_en
rest_pfas_rcom_part19_en
rest_pfas_rcom_part1_en
rest_pfas_rcom_part20_en
rest_pfas_rcom_part21_en
rest_pfas_rcom_part22_en
rest_pfas_rcom_part23_en
re